<a href="https://colab.research.google.com/github/victorgau/python_investment/blob/master/10-2%20%E5%A4%9A%E7%AD%96%E7%95%A5%E5%A4%9A%E8%82%A1%E7%A5%A8%E5%88%86%E6%9E%90.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 多策略多股票分析

In [ ]:
# 將需要的模組跟檔案放入 colab 中
!wget https://github.com/victorgau/python_investment/raw/master/backtest.py -o /dev/null
!wget https://github.com/victorgau/python_investment/raw/master/stockdata -o /dev/null
!wget https://github.com/victorgau/python_investment/raw/master/twstockdata -o /dev/null

In [ ]:
import numpy as np
import pandas as pd
import pickle

In [ ]:
# 載入計算回測指標的函式
from backtest import indicators

In [ ]:
# 用來讀入 strategies 裡面的策略
from inspect import getmembers, isfunction
import strategies

In [ ]:
filename = 'twstockdata'

In [ ]:
# 讀出預先下載好的股價資料
with open(filename, 'rb') as f:
    data = pickle.load(file=f)

# 計算各支股票的回測結果
results = []

strategy_list = [member[1] for member in getmembers(strategies) if isfunction(member[1])]

for symbol in data:
    for strategy in strategy_list:
        try:
            strategy(data[symbol])
            if np.all(data[symbol]['signals']==0):
                print("Symbol:", symbol, "使用", strategy.__name__, "策略沒有出現買賣訊號。")
                continue
            SharpeRatio, maxdd, maxddd, finalRet = indicators(data[symbol])
            days = (data[symbol].index[-1] - data[symbol].index[0]).days
            results.append((SharpeRatio, maxdd, maxddd, finalRet, days,
                            data[symbol][data[symbol]['signals'] > 0]['signals'].sum(), symbol, strategy.__name__))
        except Exception as e:
            print("Error occurs at symbol:", symbol, "Strategy:", strategy.__name__, "==>", e.args)


results_df = pd.DataFrame(results, columns=['sharpe','MaxDrawDown','MaxDrawDownDuration','returns','days', 'entries','symbol','strategy'])

In [ ]:
# Sorted by MaxDrawDown:
results_df.sort_values('MaxDrawDown',ascending=False).head()

In [ ]:
# Sorted by returns:
results_df.sort_values('returns',ascending=False).head()

In [ ]:
# Sorted by sharpe:
results_df.sort_values('sharpe',ascending=False).head()

In [ ]:
# Sorted by MaxDrawDownDuration:
results_df.sort_values('MaxDrawDownDuration',ascending=True).head()